# Setup

In [12]:
!pip install eikon
!pip install more_itertools

In [13]:
import eikon as ek
import pandas as pd
import numpy as np
import time
import os
import glob
from tqdm.auto import tqdm
from more_itertools import chunked
from math import ceil, floor

In [14]:
#"api_key_lk"
ek.set_app_key("716a396b553b441680427f612d2f89735c88bf9e")

## Screen companies

In [15]:
oil_osebx_screen = 'SCREEN(U(IN(Equity(active,public,primary))), TR.CompanyMarketCap>=500000, IN(TR.ExchangeMarketIdCode,"XOSL"), IN(TR.TRBCBusinessSectorCode,"5010","5020","5030"), CURN=USD)'
company_names = ["TR.CommonName"]#["TR.CommonName","TR.CompanyMarketCap","TR.ExchangeName","TR.TRBCBusinessSector","TR.TotalReturn3Mo"]

osbx_companies, e = ek.get_data(oil_osebx_screen, company_names)
osbx_companies = osbx_companies.set_index("Instrument")

In [16]:
oil_global_screen = 'SCREEN(U(IN(Equity(active,public,primary))), TR.CompanyMarketCap>=500000, IN(TR.TRBCBusinessSectorCode,"5010","5020","5030"), CURN=USD)'
fields_oil_global_screen = ["TR.CommonName"]

global_oil, e = ek.get_data(oil_global_screen, fields_oil_global_screen)
global_oil = global_oil.set_index("Instrument")


In [10]:
financial_global_screen = 'SCREEN(U(IN(Equity(active,public,primary))), TR.CompanyMarketCap>=500000, IN(TR.TRBCBusinessSectorCode,"5510"), CURN=USD)'
fields_financial_global_screen = ["TR.CommonName"]

global_financial, e = ek.get_data(financial_global_screen, fields_financial_global_screen)
global_financial = global_financial.set_index("Instrument")

2022-08-04 16:26:58,119 P[15024] [MainThread 15004] Error code 504 | Server Error: {"code":504,"message":"A remote server did not reply in a timely fashion. Please refer to the Data API Proxy logs for further details.","statusMessage":"Gateway Timeout"}
2022-08-04 16:26:58,119 P[15024] [MainThread 15004] HTTP request failed: EikonError-Server Error: {"code":504,"message":"A remote server did not reply in a timely fashion. Please refer to the Data API Proxy logs for further details.","statusMessage":"Gateway Timeout"}


EikonError: Error code 504 | Server Error: {"code":504,"message":"A remote server did not reply in a timely fashion. Please refer to the Data API Proxy logs for further details.","statusMessage":"Gateway Timeout"}

Now we have dataframe of all noted oil companies (with mcap > USD 5m) in eikon refinitives entire database

## Below are some interesting fields regarding public companies


In [18]:
#Market capitalization fields 
fields = ['TR.CompanyMarketCap.Date','TR.CompanyMarketCap', 'TR.PriceClose', 'TR.CompanyMarketCap.Currency'] 

#Fundamental stock data fields
profits = ['TR.TotalRevenue', 'TR.GrossProfit','TR.EBITDA','TR.EBIT', 'TR.F.NetIncAfterTax']#, 'TR.EV','MKT_CAP']
balance = ['TR.F.TotAssets','TR.F.TotCurrAssets','TR.F.TotLiab','TR.F.TotCurrLiab','TR.F.LTDebtPctofTotAssets','TR.F.STDebtPctofTotAssets',"TR.InvtrPriceToBook"]#TR.F.TotLiab(Period=FY0)
cash_flow = ['TR.F.LeveredFOCF']

fundamental_data = profits + balance + cash_flow 
reported_dates = ['TR.TotalRevenue.periodenddate']

#Company meta data fields
geography = ['TR.ExchangeMarketIdCode', 'TR.HeadquartersRegionAlt', 'TR.HeadquartersCountry', 'TR.HQStateProvince']
sectors = ['TR.TRBCEconomicSector', 'TR.TRBCBusinessSector', 'TR.TRBCIndustryGroup', 'TR.TRBCIndustry', 'TR.TRBCActivity']
founded = ['TR.OrgFoundedYear']

meta_data = geography + founded + sectors

#Broker estimates
#params_new["Period"] = "FY1"    
    
fields = ["TR.EPSMean","TR.EPSMean.periodenddate","TR.EBITMean",'TR.RevenueMean',
              "TR.ROAMean","TR.ROEMean","TR.FCFMean","TR.TotalAssets","TR.MeanPctChg(Period=FY1,WP=60d)"]

# Collect data from eikon refinitiv

In [42]:
def get_timeseries_wrapped(tickers, fields, params):
    return ek.get_timeseries(tickers, fields, start_date=params.get("SDate"), end_date=params.get("EDate"), interval=params.get("Frq", "daily")), None

In [102]:
def _decide_get_function(fields, params):
    
    timeseries_fields = ["TIMESTAMP", "VALUE", "VOLUME", "HIGH", "LOW", "OPEN", "CLOSE", "COUNT"]
    
    if isinstance(fields, str):
        fields = [fields]
    
    if fields is None or all([field in timeseries_fields for field in fields]): 
        return get_timeseries_wrapped
    
    if "Frq" in params.keys():
        if params["Frq"] in ["tick", "minute", "hour"]:
                raise ValueError("You requested data of ek.get_data() requesting timeperiods it does not support")
        
    return ek.get_data

In [105]:
def correct_fields(fields):
    if fields is None or len(fields) == 0:
        return None
    return fields

In [106]:
def correct_tickers(tickers):
    if isinstance(tickers, str):
        return [tickers]
    return tickers

In [107]:
def correct_params(chosen_function, params):
    correction = {"start_date": "SDate", "end_date":"EDate", "interval":"Frq"}
    if chosen_function == ek.get_timeseries:
        for correction_name, actual_name in correction.items():
            if actual_name in params:
                params[correction_name] = params[actual_name]
            

In [108]:
def _get_many_params(params, date_range, chosen_function):
    if date_range is not None:
        params_copies = []       
        for i in range(len(date_range)-1):
            new_params = params.copy()
            new_params["SDate"] = date_range[i].strftime('%Y-%m-%d')
            new_params["EDate"] = date_range[i+1].strftime('%Y-%m-%d')
            correct_params(new_params, chosen_function)
            params_copies.append(new_params)
        return params_copies
    return [params]

In [150]:
def _decide_tickers_and_params(chosen_function, fields, lst_of_tickers, params):
    
    data_limit = {get_timeseries_wrapped:2_500, ek.get_data:9_000}
    date_range = None
    tickers_params_pairs = []
    
    #If there this is not a timeseries problem
    if "SDate" not in params.keys():
        return [(lst_of_tickers, params)]
    
    df, err = chosen_function(lst_of_tickers[0], fields if fields is None else fields[0], params)
    fields = df.columns
    number_of_timepoints = df.shape[0]
    print(f"number_of_timepoints {number_of_timepoints}")
    
    number_of_tickers_at_once = floor(data_limit[chosen_function] / (len(fields) * number_of_timepoints))
    print(f"number_of_tickers_at_once {number_of_tickers_at_once}")
    
    if number_of_tickers_at_once < 1: 
        number_of_timepoints_at_once = floor(data_limit[chosen_function] / len(fields))
        intervals_needed = ceil(number_of_timepoints / number_of_timepoints_at_once)
        
        time_delta = (pd.to_datetime(params["EDate"]) - pd.to_datetime(params["SDate"])) / intervals_needed
        date_range = [pd.to_datetime(params["SDate"]) + i*time_delta for i in range(intervals_needed-1)] + [pd.to_datetime(params["EDate"])]
        number_of_tickers_at_once = 1
    
    sub_ticker_lst = list(chunked(lst_of_tickers, number_of_tickers_at_once))
    params_copies = _get_many_params(params, date_range, chosen_function)
    
    for sub_tickers in sub_ticker_lst:
        for params_copy in params_copies:
            tickers_params_pairs.append((sub_tickers, params_copy))
            
    
    return tickers_params_pairs

In [142]:
#Function to maximize data per http request. 
def _divide_pull_request(lst_of_tickers, fields, params):
    #batch, deduce get_data or get_timeseries size given
    #lst_of_tickers x fields x (params[start]-params[end])*freq < 10,000 or 3,000
    #also adjust params so that they fit selected function 
    tickers = correct_tickers(lst_of_tickers)
    chosen_function = _decide_get_function(fields, params)
    fields = correct_fields(fields)
    correct_params(chosen_function, params)
    tickers_params_pairs = _decide_tickers_and_params(chosen_function, fields, tickers, params)#TODO
    
    return tickers_params_pairs, chosen_function, fields

In [143]:
def wait_time(server_side_problems):
    return 3**(server_side_problems-1)

In [144]:
def handle_and_update_server_side_problem(server_side_problems, err):
    server_side_problems += 1
    print(err.message)
    seconds_to_wait = wait_time(server_side_problems)
    print(f"waiting {seconds_to_wait} second(s) before new pull request...")
    time.sleep(wait_time(seconds_to_wait))

In [145]:
def get_data_and_handle_errors(getting_function, tickers, fields, params):
    ticker_and_error = None
    
    server_side_problems = 0
    
    while server_side_problems < 5: 
        
        try:
            #if there is only one ticker causing the error should get_and_log perhaps split call into separate tickers?
            #fields not avalible should (and I think are) set to NA 
            new_df, err = getting_function(tickers, fields, params)
           
            return new_df, pd.DataFrame.from_dict({}, columns=["error"], orient="index")
        
        except ek.EikonError as err:
            if err.code ==-1:
                #Just general error not specified                        
                handle_and_update_server_side_problem(server_side_problems, err)

            if err.code == 408: 
                #HTTP Timeout exception, this just happens frequently... think it can be server side problem
                handle_and_update_server_side_problem(server_side_problems, err)
            
            if err.code == 400 or err.code == 2504: #backend error, eikon suggest waiting this one out 
                handle_and_update_server_side_problem(server_side_problems, err)
            
            if err.code == 429:
                print(err.message)
                print("If this error message keeps happening, your subscription has probably collected to much data today")
                #either daily limit is reached or call based limit reached latter can be a problem from get_datas en
        
    for ticker in tickers:
        ticker_and_error[ticker] = err 
    
    return new_df, pd.DataFrame.from_dict(ticker_and_error, columns=["error"], orient="index")

In [146]:
def save_if_criteria_met(df, filename, count):
    if (not count % 10 or count == -1) and filename is not None:
        df.to_csv(filename)

In [147]:
def format_filename(filename):
    if filename is None:
        return None
    
    if filename[-4:] == ".csv":
        return filename
    return filename + ".csv"

def format_error_filename(filename):
    if filename is None:
        return None

    if filename[-4:] == ".csv":
        return filename[:-4] + "_errors.csv"
    return filename + "_errors.csv"

In [148]:
def get_data(lst_of_tickers, fields, params, filename=None):
    #(return or save) requested data and (return or save) non-retreived-data 
    
    saved_df = None
    error_df = None
    
    filename = format_filename(filename)
    error_filename = format_error_filename(filename)
    
    tickers_params_pairs, chosen_function, fields = _divide_pull_request(lst_of_tickers, fields, params)
    
    for i, (tickers, param) in enumerate(tqdm(tickers_params_pairs)):  
        
        new_df, ticker_and_error = get_data_and_handle_errors(chosen_function, tickers, fields, param)
        
      
        if saved_df is None:
            saved_df = new_df
            error_df = ticker_and_error

        else:
            saved_df = pd.concat([saved_df, new_df])
            error_df = pd.concat([error_df, ticker_and_error])
        
        save_if_criteria_met(saved_df, filename, i)
        save_if_criteria_met(error_df, error_filename, i)
    
    save_if_criteria_met(saved_df, filename, -1)
    save_if_criteria_met(error_df, error_filename, -1)
    
    return saved_df, error_df

In [149]:
lst_of_tickers = global_oil.index.to_list()
fields = ['TR.OrgFoundedYear']

#Eikon parameters
ek_params = {}

df, errors = get_data(lst_of_tickers, fields, ek_params)
df

  0%|          | 0/1 [00:00<?, ?it/s]

,Instrument,Organization Founded Year
0,PNX.PS,2002
1,NEWH.PK,2006
2,600403.SS,1998
3,GALP.LS,1999
4,BBLS.PK,2016
...,...,...
2022,9539.SE,1997
2023,SICO.JK,2007
2024,PROE.KL,2020
2025,OPAL.OQ,2022


## Example 1) Stock and fundamental data retrival

In this example, we are interested to find Norwegian publicly traded companys fundamental data quarterly fundamental data, and its stock price, in daily format. 

To begin, we use a single ticker in the oil space, AKRBP.OL (which really did a "kule" during the spring of 2022). We find this ticker by searching for it in the eikon desktop. We are interested in its stock price possibly from 2020 and will just use get_data and hope that it hands us some interesting data without asking for specific fields. Surely, if asking for a stock, with no more information, get_data would understand that we are interested in some stock info 

In [118]:
ticker = "AKRBP.OL"
start_date = "2020-01-01"
end_date = "2022-05-06" #A very nice mans birthday
fields = None

params = {"SDate": start_date, "EDate": end_date}

In [120]:
df, err = get_data(ticker, fields, params)
df

None
None
{'SDate': '2020-01-01', 'EDate': '2022-05-06'}
number_of_timepoints 591
number_of_tickers_at_once 0


  0%|          | 0/1 [00:00<?, ?it/s]

AKRBP.OL,HIGH,CLOSE,LOW,OPEN,VOLUME
Date,,,,,
2020-01-02,293.3,289.0,287.6,288.1,580227
2020-01-03,295.8,292.9,291.4,293.6,553248
2020-01-06,297.7,296.3,292.6,297.7,513372
2020-01-07,297.1,295.7,293.0,294.9,618715
2020-01-08,297.9,295.3,294.4,297.5,672176
...,...,...,...,...,...
2022-05-02,339.7,332.2,323.4,339.7,808277
2022-05-03,343.2,343.2,323.5,331.0,771037
2022-05-04,352.1,351.0,340.9,345.0,699762


Wouldn't you know it. The get_data function gives us the daily stock data with also with intraday Volume. 

Now, we are interested in how the financials of AKER.BP has been during the same time window. Note that we below use frequency AND period the iterate through time on a fical quarter (FQ) manner, and returning the quarterly data at that time point (FQ0). NOTE THIS. If we do not specify period, get_data will return 4 copies of the yearly financial, one for each quarter.

We use the attached [TODO NAME THIS] to find some interesting fields, or we of course could use the data item browser. 

In [154]:
start_date = '2020-01-01'
end_date = '2022-04-21'
frequency = "FQ" #<-- we understand this intuitivly
period = "FQ0" #<-- How ever, this is very important to note, needed to overwrite eikons standard of returning yearly data
curn = "USD"


profit = ['TR.TotalRevenue', 'TR.GrossProfit','TR.EBITDA','TR.EBIT', 'TR.F.NetIncAfterTax']#, 'TR.EV','MKT_CAP']
balance = ['TR.F.TotAssets', 'TR.F.TotLiab']

fields = profit + balance
ek_params = {"SDate": start_date, "EDate": end_date, "Frq":frequency, "period":period, "curn":curn}

In [155]:
df, err = get_data(ticker, fields, ek_params)
df

number_of_timepoints 9
number_of_tickers_at_once 500


  0%|          | 0/1 [00:00<?, ?it/s]

,Instrument,Total Revenue,Gross Profit,EBITDA,EBIT,Net Income after Tax,Total Assets,Total Liabilities
0,AKRBP.OL,1002672000,848400000,745169000,490153000,111550000,12226786000,9859201000
1,AKRBP.OL,872105000,716062000,665503000,388091000,-334734000,11727291000,9914062000
2,AKRBP.OL,589784000,393610000,328938000,42586000,169689000,11888584000,9976500000
3,AKRBP.OL,683865000,550175000,510599000,241954000,80293000,12494548000,10565910000
4,AKRBP.OL,833508000,691440000,622691000,333283000,129467000,12420091000,10432810000
5,AKRBP.OL,1133238000,957332000,878190000,620636000,127030000,12241198000,10252205000
6,AKRBP.OL,1123754000,965519000,854535000,614162000,153811000,13075850000,11045546000
7,AKRBP.OL,1562675000,1353877000,1249866000,1003019000,205834000,13582017000,11454157000
8,AKRBP.OL,1849080000,1646706000,1558549000,1339237000,364030000,14469895000,12128004000


This is spretty fantastic! and with this our first example is complete

## Example 2) Bond data

In this example, we want to collect data for a series of bonds. We want to see what they are trading at, and find other interesting information like their maturity, and principal. 

Again, we start with a single bond and will be testing some different fields using the get_data function of this package. 

## Example 3) Macro data

In this last example, we want to collect data on some interesting macro economic varibales. We want the oil price, which could feed into example 1) and we want the interest rate in the US, which is intersting given example 2) 

This time, we start by using the eikon refinitiv desktop search tool to look up the tickers for BRT- and the US 10y rate. Again, we use the get_data function, which will under the hood use the right functionality (ek.get_timeseries) but we do not have to think about that!

In [164]:
start_date = '2020-01-01'
end_date = '2022-04-21'
macro_params = {"SDate": start_date, "EDate": end_date}

df, err = get_data(["US10YT=RR", "BRT-"], None, params)

number_of_timepoints 589
number_of_tickers_at_once 1


  0%|          | 0/2 [00:00<?, ?it/s]

In [165]:
df

,HIGH,CLOSE,LOW,OPEN,VALUE
Date,,,,,
2020-01-02,1.946,1.882,1.851,1.919,<NA>
2020-01-03,1.844,1.788,1.786,1.842,<NA>
2020-01-06,1.816,1.811,1.76,1.769,<NA>
2020-01-07,1.828,1.825,1.795,1.813,<NA>
2020-01-08,1.877,1.874,1.705,1.767,<NA>
...,...,...,...,...,...
2022-04-29,<NA>,107.28,<NA>,<NA>,107.28
2022-05-03,<NA>,105.79,<NA>,<NA>,105.79
2022-05-04,<NA>,108.74,<NA>,<NA>,108.74


In [167]:
df, err = get_data(global_oil.index[:3], profits, ek_params)

number_of_timepoints 9
number_of_tickers_at_once 500


  0%|          | 0/1 [00:00<?, ?it/s]